# CREACIÓN DE DATOS SINTÉTICOS

In [1]:
# INSTALACIÓN DE LIBRERÍAS
%pip install pandas
%pip install numpy

Note: you may need to restart the kernel to use updated packages.
Note: you may need to restart the kernel to use updated packages.


In [2]:
# IMPORTACIÓN DE LIBRERÍAS
import pandas as pd
import numpy as np
import random

In [3]:
# CONFIGURACIÓN INICIAL
np.random.seed(42)
random.seed(42)

In [13]:
# CARGAR DATASETS
df_escuelas = pd.read_csv('../fuentes_datos/datasets_finales/CSV/escuela.csv', delimiter=';')
df_semestres = pd.read_csv('../fuentes_datos/datasets_finales/CSV/semestre.csv', delimiter=';')
df_planes = pd.read_csv('../fuentes_datos/datasets_finales/CSV/plan.csv', delimiter=';')
df_estudiantes = pd.read_csv('../fuentes_datos/datasets_finales/CSV/estudiante.csv', delimiter=';')
df_cursos = pd.read_csv('../fuentes_datos/datasets_finales/CSV/curso.csv', delimiter=';')

In [17]:
# PARAMETROS BASE
escuelas = df_escuelas.set_index('id_escuela')['nombre'].to_dict()
planes = df_planes.set_index('id_escuela')['anio_inicio'].to_dict()
semestres = df_semestres['nombre'].astype(str).tolist()

In [18]:
print("DICCIONARIO ESCUELAS:")
print(escuelas)
print("\nDICCIONARIO PLANES:")
print(planes)
print("\nLISTA SEMESTRES:")
print(semestres)

DICCIONARIO ESCUELAS:
{1: 'Ingeniería de Sistemas', 2: 'Ingeniería de Software', 3: 'Ciencias de la Computación'}

DICCIONARIO PLANES:
{1: 2024, 2: 2024, 3: 2024}

LISTA SEMESTRES:
['20190', '20191', '20192', '20200', '20201', '20202', '20210', '20211', '20212', '20220', '20221', '20222', '20230', '20231', '20232', '20240', '20241', '20242', '20250', '20251', '20190', '20191', '20192', '20200', '20201', '20202', '20210', '20211', '20212', '20220', '20221', '20222', '20230', '20231', '20232', '20240', '20241', '20242', '20250', '20251', '20241', '20242', '20250', '20251']


In [19]:
# CREACIÓN DE CALIFICACIONES SINTÉTICAS
# Simular cursos tomados por estudiante (2021-2025)
cursos_tomados = []

for _, est in df_estudiantes.iterrows():
    for sem in semestres:
        if int(sem[:4]) >= est.anio_ingreso:
            for _ in range(random.randint(3, 7)):
                curso = df_cursos.sample(1).iloc[0]  # elige curso al azar
                id_curso = curso["id_curso"]
                id_ciclo = curso["id_ciclo"]
                t_aprob = curso["t_aprobacion"]  # entre 0 y 1

                # Simular si aprueba o no según tasa
                aprueba = random.random() < t_aprob

                # Nota entre 11-20 si aprueba, entre 0-10 si desaprueba
                if aprueba:
                    nota = np.clip(np.random.normal(14.5, 2), 11, 20)
                else:
                    nota = np.clip(np.random.normal(9, 2.5), 0, 10)

                id_semestre = int(sem)
                cursos_tomados.append({
                    "id_estudiante": est.id_estudiante,
                    "id_curso": id_curso,
                    "id_semestre": id_semestre,
                    "nota": int(round(nota)),
                    "id_ciclo": id_ciclo
                })


In [23]:
# GUARDAR EN UN DATAFRAME
df_cursos_tomados2 = pd.DataFrame(cursos_tomados)

In [26]:
print("DATASET CURSOS_TOMADOS 2021-2025:")
print(df_cursos_tomados2.tail(5))  # Muestra los últimos 5 registros

DATASET CURSOS_TOMADOS 2021-2025:
        id_estudiante id_curso  id_semestre  nota  id_ciclo
357518       18200108  2010103        20250    14        13
357519       18200108  2020501        20251     8        48
357520       18200108  2010205        20251     9        14
357521       18200108   201003        20251    18         2
357522       18200108  2010501        20251    12        17


In [27]:
# IDENTIFICAR CURSOS_TOMADOS CON ID_CICLO 9
df_cursos_tomados_escuela3 = df_cursos_tomados2[df_cursos_tomados2['id_ciclo'] == 9]

In [29]:
print("N REGISTROS DEL DATASET CURSOS_TOMADOS CON CICLO 9:")
print(len(df_cursos_tomados_escuela3))
print("\nDATASET CURSOS_TOMADOS CON CICLO 9:")
print(df_cursos_tomados_escuela3.head())  # Muestra los primeros 5 registros

N REGISTROS DEL DATASET CURSOS_TOMADOS CON CICLO 9:
3601

DATASET CURSOS_TOMADOS CON CICLO 9:
     id_estudiante id_curso  id_semestre  nota  id_ciclo
66        20200111   208007        20232    14         9
108       20200111   208003        20192    18         9
208       20200111   208003        20251    13         9
469       22200067   208008        20221    14         9
516       22200067   208001        20250    17         9


In [30]:
# GUARDAR EL DATASET NUEVO DE CURSOS_TOMADOS (2019-2025))
df_cursos_tomados2.to_csv('../fuentes_datos/datasets_finales/CSV/curso_tomado_completo.csv', index=False, sep=';')